In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns


import time
import math
import random
import re
import os
import sys


from utils.util import find_files_with_extension
from utils.util import  get_dataframe_from_filepath,calculate_statistics, extract_unique_mbps_and_ms
from utils.util import remove_outliers_iqr, comma_format, create_directory_if_not_exists,print_rtt_stats,save_rtt_stats


# Load data
mainpth="../data/data_2025-03-19-01-12-25"

stats_directory = os.path.join(mainpth, "Stats")
create_directory_if_not_exists(stats_directory)


folderpaths = [mainpth+'/client1_data',mainpth+'/client2_data']
siftrfilenames, siftrfilepaths, siftrfiledict = find_files_with_extension(folderpaths, '.log')


Directory already exists: ../data/data_2025-03-19-01-12-25\Stats


In [7]:
# main.py
import os
from Kernel_Data_Analysis.utils.data_preprocessing import fq_pie_pre_process_extract, l4s_pre_process_extract, trim_df
import pandas as pd

columns_to_use = [
    'queue_type', 'burst_allowance', 'drop_probability', 'current_queue_delay',
    'accumulated_probability', 'average_dequeue_time', 'length_in_bytes', 'total_drops', 'packet_length'
]

def get_stats(df, column):
    median = df[column].median()
    mean = df[column].mean()
    q25 = df[column].quantile(0.25)
    q75 = df[column].quantile(0.75)
    minimum = df[column].min()
    maximum = df[column].max()

    # Printing all the values
    print("RTT in microseconds")
    print(f"Median: {median}")
    print(f"Mean: {mean}")
    print(f"25th Percentile (Q1): {q25}")
    print(f"75th Percentile (Q3): {q75}")
    print(f"Minimum: {minimum}")
    print(f"Maximum: {maximum}")


def functionfilenameandpath(path='./', extension='.txt'):
    filepaths = []
    filenames = []

    for root, _, files_list in os.walk(path):
        for file_name in files_list:
            if file_name.endswith(extension):
                file_name_path = os.path.join(root, file_name)
                filenames.append(file_name)
                filepaths.append(file_name_path)

    return filenames, filepaths

In [8]:
kernelfilenames, kernelfilepaths = functionfilenameandpath(path=mainpth+'/kernel_data', extension='.txt')

In [9]:
kernelfilenames, kernelfilepaths, kernelfiledict = find_files_with_extension(mainpth+'/kernel_data','.txt')

In [10]:
kernelfilenames

['llmrawdata-llmused.txt',
 'llmrawdata.txt',
 'llmrawdatav1.txt',
 'llmrawdatav2.txt',
 'llmrawdatav3.txt',
 'llmrawdatav4.txt',
 'llmrawdata.txt',
 'llmrawdata-llmused.txt',
 'llmrawdata.txt',
 'llmrawdatav1.txt',
 'llmrawdatav2.txt',
 'llmrawdatav3.txt',
 'llmrawdatav4.txt',
 'llmrawdata.txt',
 'RARBG.txt',
 'NEW upcoming releases by Xclusive.txt',
 '[TGx]Downloaded from torrentgalaxy.to .txt',
 'RARBG.txt',
 'llmrawdata.txt',
 'llmrawdata.txt',
 'llmrawdata.txt',
 '1_1_fq_codel_statistics_results.txt',
 '1_1_fq_pie_statistics_results.txt',
 '1_1_l4s_8Mbps_statistics_results.txt',
 '2_2_fq_codel_statistics_results.txt',
 '2_2_fq_pie_statistics_results.txt',
 '2_2_l4s_8Mbps_statistics_results.txt',
 'requirements.txt',
 'merges.txt',
 'merges.txt',
 'LICENSE.txt',
 'LICENSE.txt',
 'merges.txt',
 'Storing experience pools here.txt',
 'train_losses.txt',
 'all result logs saved here.txt',
 'env install - ullmenv latest.txt',
 'Extra_Info.txt',
 'LLM Model Info and how to download.txt',

In [11]:
# def save_rtt_stats(filepaths, columns, output_file="rtt_stats.csv"):
#     all_stats = []

#     for filepath in filepaths:
#         if os.path.exists(filepath):
#             try:
#                 df = l4s_pre_process_extract(input_file=filepath,aqm='l4s')
#                 if all(col in df.columns for col in columns):
#                     stats = get_stats(df, columns)
#                     stats["File"] = os.path.basename(filepath)
#                     all_stats.append(stats)
#                 else:
#                     missing_cols = [col for col in columns if col not in df.columns]
#                     print(f"Missing columns {missing_cols} in {filepath}")
#             except Exception as e:
#                 print(f"Error processing {filepath}: {e}")
#         else:
#             print(f"File not found: {filepath}")

#     if all_stats:
#         stats_df = pd.DataFrame(all_stats)
#         stats_df.to_csv(output_file, index=False)
#         print(f"Statistics saved to {output_file}")
#     else:
#         print("No valid data to save.")

In [12]:
# save_rtt_stats(filepaths, [columns_to_use[3]], output_file="rtt_stats.csv")

In [13]:
from utils.util import extract_unique_mbps_and_ms
unique_combinations = extract_unique_mbps_and_ms(['./Data/kernel_data'])
print("Unique Mbps and ms Combinations:")
for mbps, ms in sorted(unique_combinations):
    print(f'{mbps} Mbps, {ms} ms')


Unique Mbps and ms Combinations:


In [14]:
unique_scenarios_dict={}
for index in range(len(kernelfilenames)):
    scenario = "_".join(kernelfilenames[index].split('_')[:5])
    if "l4s" in scenario:
        scenario = "_".join(kernelfilenames[index].split('_')[:4])
    if scenario in unique_scenarios_dict:
        unique_scenarios_dict[scenario].append(kernelfiledict[kernelfilenames[index]])
    else:
        unique_scenarios_dict[scenario]=[kernelfiledict[kernelfilenames[index]]]


In [15]:
unique_kernel_dict = {}

for scenario in unique_scenarios_dict:
    kernel_files = []
    for kernel_file in kernelfilepaths:
        if scenario in kernel_file:
            kernel_files.append(kernel_file)
    unique_kernel_dict[scenario] = kernel_files

In [16]:
unique_kernel_dict

{'llmrawdata-llmused.txt': ['.\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '.\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\All AQM LLM Data\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\All AQM LLM Data\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\All AQM LLM Data\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt'],
 'llmrawdata.txt': ['.\\Kernel_Data_Analysis\\Data\\llmrawdata.txt',
  '.\\Kernel_Data_Analysis\\Data\\old\\llmrawdata.txt',
  '.\\Kernel_Data_Analysis\\Data\\llmrawdata.txt',
  '.\\Kernel_Data_Analysis\\Data\\old\\llmrawdata

In [17]:
unique_scenarios_dict

{'llmrawdata-llmused.txt': ['/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt',
  '/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\llmrawdata-llmused.txt'],
 'llmrawdata.txt': ['/Github\\FreeBSD-EXP-Condensed\\UDP Tests\\Viz\\Kernel_Data_Analysis\\Data\\old\\llmrawdata.txt',
  '/Github\\FreeBSD-EX

In [ ]:
for filepath in kernelfilepaths:
    
    l4s_df = l4s_pre_process_extract(input_file=filepath,aqm='l4s')
    if l4s_df['current_queue_delay'].mean() > 0:
        print()
        print("*** Start ***"*3)
        print(filepath)
        print("dequeu_actions",l4s_df['dequeue_action'].value_counts())
        print("queue_type",l4s_df['queue_type'].value_counts())
        print("Classic Traffic")
        get_stats(l4s_df[l4s_df['queue_type'] == 0], columns_to_use[3])
        print("L4S Traffic")
        get_stats(l4s_df[l4s_df['queue_type'] == 1], columns_to_use[3])
        # get_stats(l4s_df, columns_to_use[3])
        print()
        print("*** END ***"*3)
    



*** Start ****** Start ****** Start ***
.\Kernel_Data_Analysis\Data\llmrawdata-llmused.txt
dequeu_actions dequeue_action
0    539070
1      3961
2       392
Name: count, dtype: int64
queue_type queue_type
0    448283
1     95140
Name: count, dtype: int64
Classic Traffic
RTT in microseconds
Median: 0.0
Mean: 88460.21374890415
25th Percentile (Q1): 0.0
75th Percentile (Q3): 160000.0
Minimum: 0
Maximum: 910000
L4S Traffic
RTT in microseconds
Median: 30000.0
Mean: 82143.7881017448
25th Percentile (Q1): 0.0
75th Percentile (Q3): 130000.0
Minimum: 0
Maximum: 910000

*** END ****** END ****** END ***

*** Start ****** Start ****** Start ***
.\Kernel_Data_Analysis\Data\llmrawdata.txt
dequeu_actions dequeue_action
0    325964
Name: count, dtype: int64
queue_type queue_type
0    247958
1     78006
Name: count, dtype: int64
Classic Traffic
RTT in microseconds
Median: 0.0
Mean: 4113.398236798167
25th Percentile (Q1): 0.0
75th Percentile (Q3): 0.0
Minimum: 0
Maximum: 590000
L4S Traffic
RTT in micr

In [ ]:
for filepath in kernelfilepaths:
    
    fq_pie_df = fq_pie_pre_process_extract(input_file=filepath,aqm='fq_pie')
    if l4s_df['current_queue_delay'].mean() < 40000 and len(l4s_df['dequeue_action'].unique()) > 1:
        print()
        print("*** Start ***"*3)
        print(filepath)
        print("dequeu_actions",l4s_df['dequeue_action'].value_counts())
        print("queue_type",l4s_df['queue_type'].value_counts())
        get_stats(l4s_df[l4s_df['queue_type'] == 0], columns_to_use[3])
        get_stats(l4s_df[l4s_df['queue_type'] == 1], columns_to_use[3])
        # get_stats(l4s_df, columns_to_use[3])
        print()
        print("*** END ***"*3)
    
